In [1]:
import re
import pandas as pd

In [2]:
###
# Get the courses from a file, into a dict.
###

courses = {}

with open('./pages/POL-2019-fall.txt', 'r') as course_file:
    # Read course file into one big string
    data = course_file.read()
    
    # Break down to courses
    data = data.split('Collapse section ')
    
    # Remove erroneous space items
    del data[0]
    
    # Key the courses into the `courses` object
    for course in data:
        courses[course[0:8]] = course

In [3]:
def handle_section(section):
    sectionCode = section[0].split('-')[0]
    
    sectionData = {
        "admin": {},
        "days": []
    }
    
    ## Handling admin stuff
    sectionData["admin"]["code"] = sectionCode
    sectionData["admin"]["type"] = section[0].split('-')[1].split('\n')[0]
    sectionData["admin"]["duration"] = section[0].split('-')[1].split('\n')[1]
    sectionData["admin"]["isOpen"] = 'Open' in section[5]
    
    ## Handling days
    
    ### Helper function to extract datetime details
    def handle_section_datetime(datetime):
        return {
            "weekday": datetime[:2],
            "time.start": datetime[3:].split(' - ')[0],
            "time.end": datetime[3:].split(' - ')[1]
        }

    ### Extract the relevant details in a very hacky way :)
    def mini_day_extractor(sectionToHandle, index):
        return {
            **handle_section_datetime(sectionToHandle[1].split('\n')[index]),
            "location": sectionToHandle[2].split('\n')[index],
            "prof": sectionToHandle[3].split('\n')[index],
            "date.start": sectionToHandle[4].split('\n')[index].split(' - ')[0],
            "date.end": sectionToHandle[4].split('\n')[index].split(' - ')[1]
        }
    
    ### Given the number of days (counted by the number of `\n` in the datetime string...), extract them
    for index in range(0, section[1].count('\n') + 1):
        sectionData["days"].append(mini_day_extractor(section, index))
            
    return {sectionCode: sectionData}

In [4]:
###
# Get the sections from each course, adding as a list to the dict item.
###

for courseCode, courseStr in courses.items():
    sections = {}
    
    if ' \n\t\t\n\t\n\t\t\n\t\n\t\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n' in courseStr:
        sectionContainer = courseStr.split(' \n\t\t\n\t\n\t\t\n\t\n\t\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')[1]
    else:
        sectionContainer = courseStr.split(' \n\t\t\n\t\n\t\t\n\t\n\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')[1]
    
    sectionData = sectionContainer.split('\n\t\n\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')
    
    for section in sectionData:
        sectionInfo = handle_section(section.split('\n\t\n'))
        
        sections.update(sectionInfo)
        
    courses[courseCode] = sections
    
## (sometimes this errors out with "list index out of range". it's all hacky anyway. power through!)

IndexError: list index out of range

In [5]:
## Clean badly processed courses :)
coursesToDelete = {}

for courseCode, courseObj in courses.items():
    if isinstance(courseObj, str):
        coursesToDelete[courseCode] = courseObj
        
for course in coursesToDelete:
    courses.pop(course, None)
        
print(str(len(coursesToDelete)) + " courses removed due to badly formed data.")

7 courses removed due to badly formed data.


In [6]:
courses

{'POL 1101': {'B00': {'admin': {'code': 'B00',
    'duration': 'FullSess.',
    'isOpen': True,
    'type': 'LEC'},
   'days': [{'date.end': '2019-12-03',
     'date.start': '2019-09-04',
     'location': '125 University (MNT) 202',
     'prof': 'Staff',
     'time.end': '21:50',
     'time.start': '19:00',
     'weekday': 'We'}]},
  'C00': {'admin': {'code': 'C00',
    'duration': 'FullSess.',
    'isOpen': True,
    'type': 'LEC'},
   'days': [{'date.end': '2019-12-03',
     'date.start': '2019-09-04',
     'location': '120 University (FSS) 2005',
     'prof': 'Staff',
     'time.end': '12:50',
     'time.start': '11:30',
     'weekday': 'Tu'},
    {'date.end': '2019-12-03',
     'date.start': '2019-09-04',
     'location': '120 University (FSS) 2005',
     'prof': 'Staff',
     'time.end': '14:20',
     'time.start': '13:00',
     'weekday': 'Fr'}]}},
 'POL 1102': {'A00': {'admin': {'code': 'A00',
    'duration': 'FullSess.',
    'isOpen': True,
    'type': 'LEC'},
   'days': [{'dat

In [7]:
## Convert the courses object to a dataframe
df = pd.DataFrame.from_dict(courses, orient='index')
df = pd.DataFrame(df.stack())

df

0
POL 1101 B00  {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         C00  {'admin': {'code': 'C00', 'type': 'LEC', 'dura...
POL 1102 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 1501 B00  {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
POL 1502 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 2101 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 2103 B00  {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         C00  {'admin': {'code': 'C00', 'type': 'LEC', 'dura...
POL 2104 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 2107 B00  {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         C00  {'admin': {'code': 'C00', 'type': 'LEC', 'dura...
POL 2156 B00  {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 2501 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 2504 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 2507 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 2556 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 3102 B00  {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         C00  {'admin': {'code': 'C00', 'type': 'LEC', 'dura...
POL 3103 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 3109 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 3113 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 3115 B00  {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
POL 3116 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 3124 B00  {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
POL 3133 B00  {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 3159 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 3162 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 3164 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
...                                                         ...
POL 6100 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 6118 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 6119 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 6120 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 6500 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 6518 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 6519 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 6520 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 6999 F00  {'admin': {'code': 'F00', 'type': 'REC', 'dura...
POL 7110 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 7112 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 7119 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 7366 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 7503 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 7511 A00  {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
POL 7513 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 7766 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 7979 F00  {'admin': {'code': 'F00', 'type': 'REC', 'dura...
POL 8110 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 8111 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 8112 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 8113 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 8510 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 8511 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 8512 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 8513 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 9200 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 9218 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 9219 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
POL 9220 A00  {'admin': {'code': 'A00', 'type': 'SEM', 'dura...

[93 rows x 1 columns]

In [8]:
## Unpack the `data` column
df = df.reset_index()
df.columns = ['course', 'code', 'data']
df

df2 = df.join(pd.io.json.json_normalize(df['data']))
df2 = df2.set_index(['course', 'code'])

df2

data admin.code  \
course   code                                                                 
POL 1101 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
         C00   {'admin': {'code': 'C00', 'type': 'LEC', 'dura...        C00   
POL 1102 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 1501 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
POL 1502 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 2101 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 2103 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
         C00   {'admin': {'code': 'C00', 'type': 'LEC', 'dura...        C00   
POL 2104 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 2107 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
         C00   {'admin': {'code': 'C00', 'type': 'LEC', 'dura...        C00   
POL 2156 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
         A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 2501 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 2504 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 2507 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 2556 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 3102 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
         C00   {'admin': {'code': 'C00', 'type': 'LEC', 'dura...        C00   
POL 3103 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 3109 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 3113 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 3115 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
POL 3116 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 3124 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
POL 3133 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
         A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 3159 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 3162 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 3164 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
...                                                          ...        ...   
POL 6100 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 6118 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 6119 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 6120 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 6500 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 6518 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 6519 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 6520 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 6999 F00   {'admin': {'code': 'F00', 'type': 'REC', 'dura...        F00   
POL 7110 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 7112 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 7119 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 7366 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 7503 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 7511 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
POL 7513 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 7766 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
POL 7979 F00   {'admin': {'code': 'F00', 'type': 'REC', 'dura...        F00   
POL 8110 A00   {'admin': {'code

In [9]:
## Unpack the `days` column

### Convert the days column from an object to a string
import ast

days_as_string = df2.astype({'days': str}).reset_index()['days'].apply(ast.literal_eval)
days_as_string

### Unpack the column. The column contains lists of objects with consistent keys, so each object becomes its own row
days_by_section = pd.concat([pd.DataFrame(x) for x in days_as_string], keys=days_as_string.index)
days_by_section

### Join the expanded rows with their original courses, dropping the now-unused columns
df3 = df2.reset_index().join(days_by_section.reset_index(1, drop=True))
df3 = df3.set_index(['course', 'code']).drop('data', 1).drop('days', 1)

df3

admin.code admin.duration  admin.isOpen admin.type    date.end  \
course   code                                                                  
POL 1101 B00         B00      FullSess.          True        LEC  2019-12-03   
         C00         C00      FullSess.          True        LEC  2019-12-03   
         C00         C00      FullSess.          True        LEC  2019-12-03   
POL 1102 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
POL 1501 B00         B00      FullSess.          True        LEC  2019-12-03   
         B00         B00      FullSess.          True        LEC  2019-12-03   
POL 1502 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
POL 2101 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
POL 2103 B00         B00      FullSess.          True        LEC  2019-12-03   
         C00         C00      FullSess.          True        LEC  2019-12-03   
         C00         C00      FullSess.          True        LEC  2019-12-03   
POL 2104 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
POL 2107 B00         B00      FullSess.          True        LEC  2019-12-03   
         B00         B00      FullSess.          True        LEC  2019-12-03   
         C00         C00      FullSess.          True        LEC  2019-12-03   
         C00         C00      FullSess.          True        LEC  2019-12-03   
POL 2156 B00         B00      FullSess.          True        LEC  2019-12-03   
         B00         B00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
POL 2501 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
POL 2504 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
POL 2507 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
...                  ...            ...           ...        ...         ...   
POL 6100 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 6118 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 6119 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 6120 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 6500 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 6518 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 6519 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 6520 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 6999 F00         F00      FullSess.          True        REC  2019-12-03   
POL 7110 A00         A00      FullSess.          True        LEC  2019-12-03   
POL 7112 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 7119 A00         A00      FullSess.          True        LEC  2019-12-03   
POL 7366 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 7503 A00         A00      FullSess.          True        LEC  2019-12-03   
POL 7511 A00         A00      FullSess.          True        LEC  2019-12-03   
POL 7513 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 7766 A00         A00      FullSess.          True        SEM  2019-12-03   
POL 7979 F00  

In [10]:
## Reorganize columns
courses_by_section_by_day = df3[[
    'admin.duration',
    'admin.isOpen',
    'admin.type',
    'prof',
    'weekday',
    'time.start',
    'time.end',
    'location',
    'date.start',
    'date.end'
]]
courses_by_section_by_day

admin.duration  admin.isOpen admin.type  \
course   code                                           
POL 1101 B00       FullSess.          True        LEC   
         C00       FullSess.          True        LEC   
         C00       FullSess.          True        LEC   
POL 1102 A00       FullSess.          True        LEC   
         A00       FullSess.          True        LEC   
POL 1501 B00       FullSess.          True        LEC   
         B00       FullSess.          True        LEC   
POL 1502 A00       FullSess.          True        LEC   
         A00       FullSess.          True        LEC   
POL 2101 A00       FullSess.          True        LEC   
         A00       FullSess.          True        LEC   
POL 2103 B00       FullSess.          True        LEC   
         C00       FullSess.          True        LEC   
         C00       FullSess.          True        LEC   
POL 2104 A00       FullSess.          True        LEC   
         A00       FullSess.          True        LEC   
POL 2107 B00       FullSess.          True        LEC   
         B00       FullSess.          True        LEC   
         C00       FullSess.          True        LEC   
         C00       FullSess.          True        LEC   
POL 2156 B00       FullSess.          True        LEC   
         B00       FullSess.          True        LEC   
         A00       FullSess.          True        LEC   
         A00       FullSess.          True        LEC   
POL 2501 A00       FullSess.          True        LEC   
         A00       FullSess.          True        LEC   
POL 2504 A00       FullSess.          True        LEC   
         A00       FullSess.          True        LEC   
POL 2507 A00       FullSess.          True        LEC   
         A00       FullSess.          True        LEC   
...                      ...           ...        ...   
POL 6100 A00       FullSess.          True        SEM   
POL 6118 A00       FullSess.          True        SEM   
POL 6119 A00       FullSess.          True        SEM   
POL 6120 A00       FullSess.          True        SEM   
POL 6500 A00       FullSess.          True        SEM   
POL 6518 A00       FullSess.          True        SEM   
POL 6519 A00       FullSess.          True        SEM   
POL 6520 A00       FullSess.          True        SEM   
POL 6999 F00       FullSess.          True        REC   
POL 7110 A00       FullSess.          True        LEC   
POL 7112 A00       FullSess.          True        SEM   
POL 7119 A00       FullSess.          True        LEC   
POL 7366 A00       FullSess.          True        SEM   
POL 7503 A00       FullSess.          True        LEC   
POL 7511 A00       FullSess.          True        LEC   
POL 7513 A00       FullSess.          True        SEM   
POL 7766 A00       FullSess.          True        SEM   
POL 7979 F00       FullSess.          True        REC   
POL 8110 A00       FullSess.          True        SEM   
POL 8111 A00       FullSess.          True        SEM   
POL 8112 A00       FullSess.          True        SEM   
POL 8113 A00       FullSess.          True        SEM   
POL 8510 A00       FullSess.          True        SEM   
POL 8511 A00       FullSess.          True        SEM   
POL 8512 A00       FullSess.          True        SEM   
POL 8513 A00       FullSess.          True        SEM   
POL 9200 A00       FullSess.          True        SEM   
POL 9218 A00       FullSess.          True        SEM   
POL 9219 A00       FullSess.          True        SEM   
POL 9220 A00       FullSess.          True        SEM   

                                        prof weekday time.start time.end  \
course   code                                                              
POL 1101 B00                           Staff      We      19:00    21:50   
         C00                           Staff      Tu      11:30    12:50   
         C00                           Staff      Fr      13:00    14:20   
POL 1102 A00                           Sta

In [11]:
## Find all courses taught by “Staff”
courses_by_section_by_day[courses_by_section_by_day['prof'].str.contains('Staff')]

admin.duration  admin.isOpen admin.type   prof weekday  \
course   code                                                          
POL 1101 B00       FullSess.          True        LEC  Staff      We   
         C00       FullSess.          True        LEC  Staff      Tu   
         C00       FullSess.          True        LEC  Staff      Fr   
POL 1102 A00       FullSess.          True        LEC  Staff      We   
         A00       FullSess.          True        LEC  Staff      Fr   
POL 1502 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      Th   
POL 2103 C00       FullSess.          True        LEC  Staff      Tu   
         C00       FullSess.          True        LEC  Staff      Th   
POL 2107 B00       FullSess.          True        LEC  Staff      Mo   
         B00       FullSess.          True        LEC  Staff      Th   
         C00       FullSess.          True        LEC  Staff      Tu   
         C00       FullSess.          True        LEC  Staff      Th   
POL 2156 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      We   
POL 3109 A00       FullSess.          True        LEC  Staff      Mo   
POL 3113 A00       FullSess.          True        LEC  Staff      Tu   
POL 3115 B00       FullSess.          True        LEC  Staff      Th   
POL 3124 B00       FullSess.          True        LEC  Staff      Th   
POL 3133 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      We   
POL 3175 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      Th   
POL 3177 A00       FullSess.          True        LEC  Staff      Fr   
POL 3178 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      Th   
POL 3370 A00       FullSess.          True        LEC  Staff      We   
         A00       FullSess.          True        LEC  Staff      Fr   
POL 3513 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      We   
POL 3535 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Fr   
POL 3545 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Th   
POL 3559 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      Th   
POL 3564 A00       FullSess.          True        LEC  Staff      We   
         A00       FullSess.          True        LEC  Staff      Fr   
POL 3770 A00       FullSess.          True        LEC  Staff      Mo   
POL 4134 A00       FullSess.          True        LEC  Staff      Mo   
POL 4163 A00       FullSess.          True        LEC  Staff      Mo   
POL 4173 A00       FullSess.          True        LEC  Staff      Th   
POL 4189 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Fr   
POL 4350 A00       FullSess.          True        SEM  Staff      Th   
POL 4527 A00       FullSess.          True        LEC  Staff      Th   
POL 4541 A00       FullSess.          True        LEC  Staff      Th   
POL 6999 F00       FullSess.          True        REC  Staff      TB   
POL 7979 F00       FullSess.          True        REC  Staff      TB   

              time.start time.end                        location  date.start  \
course   code                                                                   
POL 1101 B00       19:00    21:50        125 University (MNT) 202  2019-09-04   
         C00       11:30    12:50       120 University (FSS) 2005  2019-09-04   
         C00     